In [1]:
#!/usr/bin/env python3
"""logistic_regression
James Gardner 2019

performs logistic regression on feature vectors
against positional matching labels
using pytorch
"""

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
# Step 1. Load Dataset
# Step 2. Make Dataset Iterable
# Step 3. Create Model Class
# Step 4. Instantiate Model Class
# Step 5. Instantiate Loss Class
# Step 6. Instantiate Optimizer Class
# Step 7. Train Model

In [ ]:
catalogue = pd.read_csv('patch_catalogue.csv')
catalogue.set_index(['name_TGSS','name_NVSS'],inplace=True)

scores = catalogue['score']
del (catalogue['ra_TGSS'],catalogue['dec_TGSS'],
     catalogue['ra_NVSS'],catalogue['dec_NVSS'],
     catalogue['score'])

catalogue['log_flux_TGSS']       = np.log10(catalogue['peak_TGSS'])
catalogue['log_integrated_TGSS'] = np.log10(catalogue['integrated_TGSS'])
catalogue['log_ratio_flux_TGSS'] = np.log10(catalogue['peak_TGSS']/
                                            catalogue['integrated_TGSS'])
catalogue['log_flux_NVSS']       = np.log10(catalogue['peak_NVSS'])

In [ ]:
labels = (scores.values > 0.1)
features = catalogue.values

features = Variable(torch.Tensor(???))
labels = Variable(torch.Tensor(???))

In [8]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = F.sigmoid(self.linear(x))
        return outputs

In [ ]:
input_dim  = 1
output_dim = 1
lr_rate = 0.01

# batch_size = 100
# n_iters    = 3000
# num_epochs = n_iters / (len(x_data) / batch_size)
num_epochs = 100

In [9]:
model = LogisticRegression(input_dim, output_dim)
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

/home/james/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
for epoch in range(num_epochs):
    # reset gradient accumulation
    optimizer.zero_grad()
    # forward step
    predictions = model(features)
    loss = criterion(predictions, labels)
    # backwards step
    loss.backward()
    optimizer.step()

In [ ]:
weights = ???

In [ ]:
plt.figure(figsize=(14,7))
plt.rcParams.update({'font.size': 18})
plt.bar(range(len(weights)),weights)
plt.xlabel('weights')
plt.xticks(range(len(weights)),catalogue.columns,rotation='vertical')
plt.ylabel('co-eff')
plt.title('torch_lr')
plt.savefig('torch_lr.pdf',bbox_inches='tight') 
# plt.close()

In [ ]:
# model = LinearRegression()
# criterion = torch.nn.MSELoss(size_average=False)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)